# Bağlantı

In [ ]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 160772 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.26-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.26-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.26-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [ ]:
!mkdir -p drive
!google-drive-ocamlfuse drive
!ls

adc.json  drive  sample_data


# Veri Seti Yükleme

In [ ]:
import pandas as pd
import numpy as np
# csv den veri yükleme

from openpyxl import Workbook,load_workbook 
yorumlar = pd.read_excel('/content/drive/Colab Notebooks/trendyolveri/trendyolverislenmis.xlsx')
#data = pd.read_csv('/content/drive/Colab Notebooks/hepsiburadaveri/reviews2.txt', delimiter='\t')

#data = data.sample(frac=1).reset_index(drop=True)

In [ ]:
#filitreleme büyük harf/ küçük harf
import re
import nltk
# gereksiz kelimler çıkarma
from nltk.stem.porter import PorterStemmer
# ps ingilizce için türkçede deniyeceğim
ps = PorterStemmer()
nltk.download('stopwords')
from nltk.corpus import stopwords
yorumdizim=[]
for i in range(98): # execelde kolon uzunluğu 98 di
    # Yorumlar bizim verilerde ki başlık
    data = re.sub('[^a-zA-Z]',' ',yorumlar['Yorumlar'][i])
    # Küçük harfe çevirme
    data = data.lower()
    # listeye çevirme
    data=data.split()
    # Önemli set ile küme yapıyoruz bunun nedeni aynı ifade birden fazla geçmemesi için
    # eğer kelime listede yoksa stemle yani listenin ilk elemanı yap
    data = [ps.stem(kelime) for kelime in data if not kelime in set(stopwords.words('turkish'))]
    # şunu dene olmazsa
    # data = [yorum.stem(kelime) for kelime in data ]
    # data değişkenini boşluklarla birleştir
    data = ' '.join(data)
    yorumdizim.append(data)
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=2000)
#bağımsız değişken
X = cv.fit_transform(yorumdizim).toarray()
# bağımlı değişken
y = yorumlar.iloc[:,1].values # [:,1] bütün satırlar ve 1. kolon
#from sklearn.cross_decomposition import train_test_split
from sklearn.model_selection import train_test_split
# Test ve eğitim için bölme
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)
from sklearn.naive_bayes import MultinomialNB
gnb= MultinomialNB()
#X_test_counts = count_vect.transform(X_test)
#X_test_tfidf = tfidf_transformer.transform(X_test_counts)

##from sklearn.naive_bayes import GaussianNB
##gnb = GaussianNB()
# X_train den y_train i öğren diyeceğiz
gnb.fit(X_train,y_train) # fit öğretmek için kullanılır
# X_test tahmin et diyeceğiz ve y_test ile karşılaştıracağız
y_pred = gnb.predict(X_test) #predict tahmin için kullandık
# şimdide gerçek verilerle karşılaştır diyeceğiz
from sklearn.metrics import confusion_matrix
# başarı oranına bakacağız
cm = confusion_matrix(y_test, y_pred)
print(cm)



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[[0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0

In [ ]:
data.head() #ilk beş girdi görmek için

,beklentimin altında bir ürün kaliteli değil
0,ford focus 2 aracıma tam oldu fakat ip kısa ka...
1,"gelen ürün görseldeki gibi değildi,beyaz makya..."
2,ürün güzel şık. resimdeki renkler ile ilgisi y...
3,arka koltuk icin ford fiestaya eni dar geldi.
4,ürün yeni aldım. birkaç kez kullandım. yumurta...


Metinlerin pozitif, negatif ve nötr durumları

In [ ]:
data['beklentimin altında bir ürün kaliteli değil'].value_counts()


fena değil                                                                                                                                                                  3
fena deyil                                                                                                                                                                  3
fena degil                                                                                                                                                                  2
iyi                                                                                                                                                                         2
güzel                                                                                                                                                                       2
                                                                                                                                  

# Veri Seti Önişleme

In [ ]:
import string #noktalama işaretleri
import re
import nltk #etkisiz kelimeler
from nltk.corpus import stopwords
nltk.download('stopwords') # önemliiiii
noktalama = string.punctuation
etkisiz = stopwords.words('turkish')
print(noktalama)
print(etkisiz)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['acaba', 'ama', 'aslında', 'az', 'bazı', 'belki', 'biri', 'birkaç', 'birşey', 'biz', 'bu', 'çok', 'çünkü', 'da', 'daha', 'de', 'defa', 'diye', 'eğer', 'en', 'gibi', 'hem', 'hep', 'hepsi', 'her', 'hiç', 'için', 'ile', 'ise', 'kez', 'ki', 'kim', 'mı', 'mu', 'mü', 'nasıl', 'ne', 'neden', 'nerde', 'nerede', 'nereye', 'niçin', 'niye', 'o', 'sanki', 'şey', 'siz', 'şu', 'tüm', 've', 'veya', 'ya', 'yani']


In [ ]:
for d in data['beklentimin altında bir ürün kaliteli değil'].head():
  print(d+ '\n------------------------------')
  # Etkisiz kelimeleri atmak
  temp= ''
  for word in d.split():
    if word not in etkisiz and not word.isnumeric():
        temp += word + ''
  print(temp+ '\n********************')

ford focus 2 aracıma tam oldu fakat ip kısa kalıyor.şu ipi 1-2 metre daha uzun yollayın.yarım kaldı ve aynı ipten bulamıyorum bir türlü.dolayısı ile işe yaradı diyemiyorum.
------------------------------
fordfocusaracımatamoldufakatipkısakalıyor.şuipi1-2metreuzunyollayın.yarımkaldıaynıiptenbulamıyorumbirtürlü.dolayısıişeyaradıdiyemiyorum.
********************
gelen ürün görseldeki gibi değildi,beyaz makyaj pamuğu kadar bir top ve ucunda 3 tane ipten ibaretti.
------------------------------
gelenürüngörseldekideğildi,beyazmakyajpamuğukadarbirtopucundataneiptenibaretti.
********************
ürün güzel şık. resimdeki renkler ile ilgisi yok koyu renk ve resimdekilerden daha güzel. siyahı tavsiye ederim.
------------------------------
ürüngüzelşık.resimdekirenklerilgisiyokkoyurenkresimdekilerdengüzel.siyahıtavsiyeederim.
********************
arka koltuk icin ford fiestaya eni dar geldi.
------------------------------
arkakoltukicinfordfiestayaenidargeldi.
********************
ürün yeni aldı

Noktalama işaretleri silme

In [ ]:
for d in data['beklentimin altında bir ürün kaliteli değil'].head():
  print(d+ '\n------------------------------')
  # Noktalama işaretleri atma
  temp= ''
  for word in d:
    if word not in noktalama:
        temp += word
  print(temp+ '\n********************')
  d= temp

ford focus 2 aracıma tam oldu fakat ip kısa kalıyor.şu ipi 1-2 metre daha uzun yollayın.yarım kaldı ve aynı ipten bulamıyorum bir türlü.dolayısı ile işe yaradı diyemiyorum.
------------------------------
ford focus 2 aracıma tam oldu fakat ip kısa kalıyorşu ipi 12 metre daha uzun yollayınyarım kaldı ve aynı ipten bulamıyorum bir türlüdolayısı ile işe yaradı diyemiyorum
********************
gelen ürün görseldeki gibi değildi,beyaz makyaj pamuğu kadar bir top ve ucunda 3 tane ipten ibaretti.
------------------------------
gelen ürün görseldeki gibi değildibeyaz makyaj pamuğu kadar bir top ve ucunda 3 tane ipten ibaretti
********************
ürün güzel şık. resimdeki renkler ile ilgisi yok koyu renk ve resimdekilerden daha güzel. siyahı tavsiye ederim.
------------------------------
ürün güzel şık resimdeki renkler ile ilgisi yok koyu renk ve resimdekilerden daha güzel siyahı tavsiye ederim
********************
arka koltuk icin ford fiestaya eni dar geldi.
------------------------------
a

Verileri kayıt etme

In [ ]:
data.to_csv(r'/content/drive/Colab Notebooks/Hepsiburadaveri/cleaned.csv',index=False)


# Arındılımış veri yükleme

In [ ]:
import pandas as pd
data = pd.read_csv('/content/drive/Colab Notebooks/Hepsiburadaveri/cleaned.csv', sep=",", names=['review', 'label'])
print(data.head())

                                              review  label
0                                             review  label
1        beklentimin altında bir ürün kaliteli değil    NaN
2  ford focus 2 aracıma tam oldu fakat ip kısa ka...    NaN
3  gelen ürün görseldeki gibi değildi,beyaz makya...    NaN
4  ürün güzel şık. resimdeki renkler ile ilgisi y...    NaN


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 100)


# Veri Setini Bölme

In [ ]:
# Arındırılmış veriyi train ve test kümelerine ayırıyoruz
from sklearn.model_selection import train_test_split
# Eğitim ve test olarak ayıracağız. train=eğitim, test=test
X_train, X_test, y_train, y_test = train_test_split(data['review'].values.astype('U'),data['label'].values.astype('U'), test_size=0.1, random_state=42)
print(X_train.shape)
#print(Xtest.shape)                                                                      

(13392,)


# Sayma Vektörü Oluşturma

In [ ]:
# Train kümesindeki cümlelerin sayma vektöelerini çıkarıyoruz
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
print(X_train_counts.shape)

(13392, 42433)


Tf*Idf vektörü oluşturma
sadece o yoruma özgü(temsil edecek kelimler olmalı)

In [ ]:
# Train kümesindeki cümlelerin TF*IDF vektörlerini sayma vekttörlerinden oluşturuyoruz
# Tf= terim sıklığı, IDF= bir kelimenin dökümanda kaç kere geçtiği
# TF*IDF bütün olarak; bir kelimenin bir döküman içinde ki önemini gösterir
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
print(X_train_tfidf.shape)

(13392, 42433)


# Naive Bayes Model Eğitimi

In [ ]:
# Çok modlu Naive Bayes Sınıflandırıcısı eğitiyoruz
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, y_train)
X_test_counts = count_vect.transform(X_test)
X_test_tfidf = tfidf_transformer.transform(X_test_counts)

# Model Performansı Ölçme

In [ ]:
# Sınıflandırıcı ile test seti üzerindeki tahminleme yapıyoruz
y_pred = clf.predict(X_test_tfidf)
for review, sentiment in zip(X_test[:5], y_pred[:]):
  print('%r => %s' % (review, sentiment))

'yine de hayatınızdaki özel kişi için güzel bir koku sunarsınız... parfümle yanınınzdan geçen insanları etklieyemeyi düşünmeyin..kokusu hafif..' => nan
'cam kalitesi çok düşük. .çizikler çok rahatsız ediyor' => nan
'cihaz zamanında elime ulaştı ve ıphone telefonumu 6 defa şarj edebiliyorum ancak cihaz diğer cihazlara göre oldukça ağır bunu göz önünde bulundurmanızı tavsiye ediyorum.' => nan
'kurulumu basit. suntaları kalitesiz. derinliği çok az, içine takım elbise astığınızda kapıyı zorluyor. ufak bir dolap, rengi görseldeki gibi bej değil bildiğiniz bembeyaz banyo dolabı gibi. öğrenciler ya da bayanlar için ideal olabilir.' => nan
'bildirim ışığı yok telefonun ekranı açmazsanız bildirim gelip gelmediğini anlama ihtimaliniz yok.' => nan


Test Sonuçları

In [ ]:
# Performans sonuçları
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

1.0
